### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

In [13]:
ratings = pd.read_csv('ratings.csv')

In [16]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [26]:
def get_max_min_time_diff(df_temp, user_rating_count=100):  
    df_temp = df_temp.groupby('userId').agg(['min', 'max', 'count'])['timestamp'].reset_index()
    df_temp['timestamp_delta'] = df_temp['max'] - df_temp['min']
    return df_temp[df_temp['count'] >= user_rating_count]

In [27]:
get_max_min_time_diff(ratings).head()

,userId,min,max,count,timestamp_delta
3,4,949778714,949982274,204,203560
4,5,1163373044,1163375145,100,2101
7,8,1154389340,1154474527,116,85187
14,15,997937239,1469330735,1700,471393496
16,17,1127468587,1127476640,363,8053


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


#### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [28]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [29]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [30]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [31]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [37]:
def get_all_client_directions(with_addr=True):
    df_all_dir = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)
    if with_addr:
        return client_base.merge(df_all_dir, how='left').fillna(0)
    else:
        return df_all_dir

##### With client's address

In [40]:
get_all_client_directions()

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


##### Without client's address

In [41]:
get_all_client_directions(False)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Для связи покупки и/или визита с координатами можно использовать подход:
    * Выбор ближайшего по timestamp местоположиню пользователя (дата визита/покупки и дата получения местоположения)
    * Завести адресник с местами визитов/покупок, в котором будут координаты этого места и забирать данные о ближайших координатах к этому месту визита/покупки + адекватная оценка по времени координат и визитов
2. Насчет удаленности координат, имеет смысл предварительно почистить гео-данные, они периодически могут давать ошибочное положение, если одна-две координаты в один и тот же день показывают очень отдаленные значения (порог отдаленности может зависеть от населенного пункта или меняться по оценке аналитика), то имеет смысл их вообще не учитывать в анализе визитов/покупок, но отдать на анализ по точности гео трекинга
3. Из доп.признаков - как много и часто пользователь перемещается, какие места посещает, можно ли эти места как-то привязать к его покупкам (реклама, акции и т.д.), также да данным можно предположить способ передвижения - от этого опять-таки может зависеть конкретное предложение.